# Reactor simulation code - ChBE 202 - Fall 2022

Includes portions for both Batch Reactor and Packed Bed Reactor (PBR) models. Batch integrated with respect to time ($t$), PBR integrated with respect to catalyst contact time ($\tau$, inlet gas molar flow rate divided by catalyst mass).

### Functions:
 - **diff_batch(U,t,*inputs)** : calculates array dU of differential equations; to be used with ODE integrator
 - **diff_PBR
(U,tau,*inputs)** : calculates array dU of differential equations; to be used with ODE integrator
 - **rate_eqns(ai,pars)** : calculates reaction rates based on input concentrations and rate parameters using functional forms specified in the function; rates reported are on a per catalyst amount (e.g., mass) basis ; used mainly in conjunction with 'diff' functions
 - **batch_predict(x_df,*pars)** : predicts concentrations of species in "fitted_species" list based on input conditions dataframe (x_df) and kinetic parameters ; used mainly for fitting (i.e., curve_fit) or to develop parity plots
 - **log_batch_predict** : simply takes the natural logarithm of the batch_predict outputs; used for data fitting when it is desirable to evaluate residuals of logs instead of exact numbers to dampen data extremes
 - **PBR_predict(x_df,*pars)** : PBR version of batch_predict
 - **log_PBR_predict(x_df,*pars)** : PBR version of log_batch_predict
 - **parity_plot(exp_df,model_df)** : generate parity plot from dataframes of experimental and model data

### Evaluation blocks:
- ***Import packages***: imports all packages required to run
- ***Import data***: imports data from a specified Excel file
- ***Integrate batch reactor and plot data***: uses the functions to simulate a batch reactor based on specified initial conditions and rate parameters
- ***Batch data fitting***: fits parameters based on input data; must match the fitting parameters with those speciufied in the "rate_eqns" function if you are only fitting a subset of parameters
- ***Integrate PBR and plot data***: uses the functions to simulate a PBR based on specified initial conditions and rate parameters
- ***PBR data fitting***: fits parameters based on input data; must match the fitting parameters with those speciufied in the "rate_eqns" function if you are only fitting a subset of parameters

In [ ]:
########## Import packages ##########
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats.distributions import t as tdist
import datetime
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

Rlatm = 0.082057

print('Last cell eval:', datetime.datetime.now().strftime("%H:%M on %Y-%m-%d"))

In [ ]:
### Differential equations for a batch reactor
def diff_batch(t,U,*inputs):
  
    ### Sort input conditions from parameters   
    conditions = inputs[0]
    pars = inputs[1]
    
    ### Sort conditions
    T    = conditions[0]
    V0   = conditions[1]
    mcat = conditions[2]
    C0   = conditions[3]
    
    # Relate U to Nj
    Nj = U
    
    ### Set all negative values to zero to avoid errors
    Nj[Nj<0] = 0

    # Volume calculation
    V = V0 # Assumes constant volume
    
    # Concentration
    Cj = Nj / V
    
    # Calculate rate equations
    ri = rate_eqns(Cj,pars,T)

    # Calculate net generation rates of each species
    R = np.dot(stoich.T,ri)
    
    # Calculate differential variable dU=dNi
    dU = R * mcat
    
    # Reorganize list to fit required dimensions 
    dU = np.array(dU).tolist()
    
    return dU

In [ ]:
def diff_PBR(W,U,*inputs):
  
    ### Sort input conditions from parameters   
    initial_conditions = inputs[0]
    par = inputs[1]
    
    T0  = initial_conditions[0]
    P0  = initial_conditions[1]
    v0  = initial_conditions[2]
    Fj0  = initial_conditions[3]
    
    ### Relate U to Fj
    T = U[0]
    gamma = U[1]
    P = gamma * P0
    Fj = U[2:]
    
    ### Set all negative values to zero to avoid errors
    Fj[Fj<0] = 0
    
    F = np.sum(Fj)

    # Concentration
    Pj = Fj / F * P
    Cj = Pj / 82.057 / T
    
    # Calculate rate equations
    ri = rate_eqns(Cj,par,T)

    # Calculate net generation rates of each species
    R = np.dot(stoich.T,ri)
    
    # Calculate differential variable dFj=dFj/dV
    dFj = R
    
    # Calculate change in temperature and pressure drop
    dT = 0 # Assume isothermal
    dgamma = 0 # Assume no pressure drop
    
    # Populate dependent variable differential array dU
    dU = np.zeros(len(U))
    dU[0]  = dT
    dU[1]  = dgamma
    dU[2:] = dFj
    
    # Reorganize list to fit required dimensions 
    dU = np.array(dU).tolist()
    
    return dU

In [ ]:
### Rate equations for acetone coupling
def rate_eqns(aj,pars,T):

    Cj = aj
    ri = np.zeros(nrxns)

    p = 0
    k1        = pars[p] ; p = p+1 
    KC1       = pars[p] ; p = p+1
    k2        = pars[p] ; p = p+1
    KC2       = pars[p] ; p = p+1
    Kads_H2O  = pars[p] ; p = p+1
    
    ### Reaction quotients
    QC1 = Cj[IDN['diacetone alcohol']]/Cj[IDN['acetone']]**2
    
    if Cj[IDN['diacetone alcohol']]==0:
        QC2 = 0
    else:
        QC2 = Cj[IDN['mesityl oxide']]*Cj[IDN['water']]/Cj[IDN['diacetone alcohol']]
    
    ### Rate expressions
    ri[0] = k1 * Cj[IDN['acetone']]**2 * (1-QC1/KC1) / (1+Kads_H2O*Cj[IDN['water']])**2
    ri[1] = k2 * Cj[IDN['diacetone alcohol']] * (1-QC2/KC2) / (1+Kads_H2O*Cj[IDN['water']])
        
    return ri

In [ ]:
### Predict concentrations of desired species based on input reactor conditions
def batch_predict(x_df,*pars):
    
    ### For every datapoint, integrate differential equations and report key outputs
    output_matrix = np.zeros([ndpts,len(fitted_species)])
    for i in range(ndpts):
        temp = x_df['Temperature'][i]
        V0 = x_df['Feed Volume'][i] / 1000 # mL --> L
        mcat = x_df['Catalyst Mass'][i]
        C0 = C0_df.iloc[i].to_numpy('float')
        tf = x_df['Reaction Length'][i]
               
        conditions = [temp,V0,mcat,C0]
        inputs = [conditions,pars]
        
        ### Inlet moles
        N0 = C0 * V0

        ### Determine integration span and spacing
        tlim = (0,tf)
        tlist = np.linspace(tlim[0],tlim[1],num=101) # Not essential
    
        ### Integrate
        solution = solve_ivp(diff_batch,tlim,N0,t_eval=tlist,args=inputs,method='Radau')
        t = solution.t
        Nj = solution.y.T
        Njf = Nj[-1,:]
        Vf = V0 # Assumes constant volume
        Cjf = Njf / Vf
        
        ### Select concentrations key to fitting
        C_outputs = np.zeros(len(fitted_species))
        
        for j in fitted_species:
            C_outputs[fitted_species.index(j)] = Cjf[IDN[j]]
            
        output_matrix[i,:] = C_outputs
    
    output = np.reshape(output_matrix,np.prod(output_matrix.shape))
    
    return output

In [ ]:
### Use to fit log of outputs instead
def log_batch_predict(x_df,*pars):
    original = batch_predict(x_df,*pars)
    log_data = np.log(original)
    
    return log_data

In [ ]:
### Predict concentrations of desired species based on input reactor conditions
def PBR_predict(x_df,*pars):
    
    ### Use inputs_dfs defined outside of program as global variables
    conditions_df = x_df.iloc[:,:4]
    yj0_df        = x_df.iloc[:,4:]
    
    ### For every datapoint, integrate differential equations and report key outputs
    output_matrix = np.zeros([ndpts,len(fitted_species)])
    for i in range(ndpts):
        T0 = conditions_df['Temperature'][i]
        v0 = conditions_df['Feed Volumetric Flow Rate'][i] / 1000 # sccm --> slpm
        W = conditions_df['Catalyst Mass'][i]
        P0 = conditions_df['Pressure'][i]
        yj0 = yj0_df.iloc[i].to_numpy('float')

        ### Inlet gas flow rate
        F0 = 1 * v0 / 0.08314 / 273.15 # moles/min with STP = 1 bar, 273.15 K

        ### Inlet moles
        Fj0 = yj0 * F0

        ### Determine integration span and spacing
        tlim = (0,W)

        ### Inputs
        conditions = [T0,P0,v0,Fj0]    
        inputs = [conditions,pars]

        ### Initial conditions
        U0 = [T0,1]+Fj0.tolist()

        ### Integrate
        solution = solve_ivp(diff_PBR,tlim,U0,args=inputs,method='Radau')
        W = solution.t
        U = solution.y.T
        Fjf = U[-1,2:]            
        yjf = Fjf / sum(Fjf)
        
        ### Select concentrations key to fitting
        y_outputs = np.zeros(len(fitted_species))
        
        for j in fitted_species:
            y_outputs[fitted_species.index(j)] = yjf[IDN[j]]
            
        output_matrix[i,:] = y_outputs
    
    output = np.reshape(output_matrix,np.prod(output_matrix.shape))
    
    return output

In [ ]:
### Use to fit log of outputs instead
def log_PBR_predict(x_df,*pars):
       
    original = PBR_predict(x_df,*pars)
    log_data = np.log(original)
    
    return log_data

In [ ]:
### Generate parity plot
def parity_plot(exp_df,model_df):
    
    ### Interpret inputs
    species_list = exp_df.columns.to_list()
    nspecies = len(species_list)
    
    parity_df = pd.DataFrame(columns=fitted_species)

    for i in species_list:
        parity_df[i] = [min([exp_df[i].min(),model_df[i].min()]),max([exp_df[i].max(),model_df[i].max()])]

    ### Export data to excel
    with pd.ExcelWriter('parity.xlsx') as writer:  
        exp_df.to_excel(writer, sheet_name='Experiment')
        model_df.to_excel(writer, sheet_name='Model')
        
    ### Plot data and create output parity daframe
    fig, axs = plt.subplots(1, nspecies,figsize=(6, 3), dpi=300)
    
    if nspecies > 1:
        for j in species_list:
            n = species_list.index(j)
            axs[n].plot(exp_df[j],model_df[j],'ob',mfc='none')
            axs[n].plot(parity_df[j],parity_df[j],'--b')
            axs[n].set_xlabel('Experiment')
            axs[n].set_ylabel('Prediction')
            axs[n].set_title(j)
            
    elif nspecies == 1:
            j = species_list[0]
            axs.plot(exp_df[j],model_df[j],'ob',mfc='none')
            axs.plot(parity_df[j],parity_df[j],'--b')
            axs.set_xlabel('Experiment')
            axs.set_ylabel('Prediction')
            axs.set_title(j)
    
    ### Display results
    fig.tight_layout()
    plt.show()
    
    return fig,axs

In [ ]:
########## Import data ##########
print('Last cell eval:', datetime.datetime.now().strftime("%H:%M on %Y-%m-%d"))

input_file = 'Batch_Inp_Acetone_Example.xlsx'

### Species information
species_df = pd.read_excel(input_file, sheet_name = 'Species')
species_df = species_df.set_index('Species Name')
IDN = species_df['IDN']
nspecies = len(species_df)
all_species = species_df.index.to_list()

### Reaction information
rxns_df = pd.read_excel(input_file, sheet_name = 'Reactions')
nrxns = rxns_df.index.size
rxns_df = rxns_df.fillna(0)
rxnIDN = rxns_df[['Rate constant','RxnIDN']].set_index('Rate constant')['RxnIDN']

### Generate stoichiometric matrix
stoich_names = rxns_df[['R1','R2','R3','R4','P1','P2','P3','P4']].to_numpy()
stoich_input = rxns_df[['R1stoich','R2stoich','R3stoich','R4stoich','P1stoich','P2stoich','P3stoich','P4stoich']].to_numpy()
stoich = np.zeros([nrxns,nspecies])
for i in range(nrxns):
    for j in range(len(stoich_names[0,:])):
        if type(stoich_names[i,j]) == str:
            stoich[i,IDN[stoich_names[i,j]]] = stoich_input[i,j]

In [ ]:
########## Integrate batch reactor and plot data ##########
print('Last cell eval:', datetime.datetime.now().strftime("%H:%M on %Y-%m-%d"))

### Define constants
k1 = 0.005
KC1 = 0.003
k2 = 0.00
KC2 = 1000
Kads_H2O = 10

pars = [k1,KC1,k2,KC2,Kads_H2O]

### Initial temperature
T0 = 35 + 273 # K ; May or not be used by code

### Initial volume
V0 = 60 / 1000 # mL --> L

### Catalyst mass
mcat = 0.1

### Initial concentrations
C0 = np.zeros(nspecies)
C0[IDN['acetone']]           = 13.5 # mol / L
C0[IDN['water']]             = 0.0 # mol / L

### Initial moles
N0 = C0 * V0

### Determine integration span and spacing
tlim = (0,6)
tlist = np.linspace(tlim[0],tlim[1],num=101) # specify number of evenly-spaced points to report

### Inputs
conditions = [T0,V0,mcat,C0]    
inputs = [conditions,pars]

### Integrate
solution = solve_ivp(diff_batch,tlim,N0,t_eval=tlist,args=inputs,method='Radau')
t = solution.t
Nj = solution.y.T
V = V0 # Assumes constant volume
Cj = Nj / V

### Convert lists to numpy arrays
t = np.array(t)
Cj = np.array(Cj)
tau = t / mcat # Reaction time normalized by catalyst mass

### Conversion, yield, selectivity
X = (N0[IDN['acetone']]*np.ones(len(Nj[:,0]))-Nj[:,IDN['acetone']])/N0[IDN['acetone']]

Y_DAA = 2*(Cj[:,IDN['diacetone alcohol']]-C0[IDN['diacetone alcohol']])/C0[IDN['acetone']]
Y_MO = 2*(Cj[:,IDN['mesityl oxide']]-C0[IDN['mesityl oxide']])/C0[IDN['acetone']]

S_DAA = Y_DAA / X
S_MO = Y_MO / X

### Define pandas dataframe with outputs
soln = pd.DataFrame(t,columns = ['Time (h)'])
soln[['Conversion']] = pd.DataFrame(X)
soln[species_df.index.to_numpy()] = pd.DataFrame(Cj)
soln[['DAA Yield']] = pd.DataFrame(Y_DAA)
soln[['MO Yield']] = pd.DataFrame(Y_MO)
soln[['DAA Sel']] = pd.DataFrame(S_DAA)
soln[['MO Sel']] = pd.DataFrame(S_MO)

### Save dataframe to a csv
soln.to_csv('Batch_model_output.csv')

### Report dataframe
print('\n',soln,'\n')

### Plot data
fig = plt.figure()
ax1 = plt.subplot(231)
ax2 = plt.subplot(232)
ax3 = plt.subplot(233)

ax1.plot(t,Cj[:,IDN['acetone']],'b')
ax1.plot(t,Cj[:,IDN['diacetone alcohol']],'-g')
ax1.plot(t,Cj[:,IDN['mesityl oxide']],'-.m')
ax1.set_xlabel('t (h)')
ax1.set_ylabel('C (mol/L)')
ax1.legend(('A','DAA','MO'))

ax2.plot(t,X,'g')
ax2.set_xlabel('t (h)')
ax2.set_ylabel('Conversion')

ax3.plot(X,S_DAA,'-g')
ax3.plot(X,S_MO,'-.m')
ax3.legend(['DAA','MO'])
ax3.set_xlabel('Conversion')
ax3.set_ylabel('Selectivity')

fig.tight_layout()
plt.show()
print('Conversion at ',max(t),'h is ','{0:,.3f}'.format(X[-1]*100),'%')

In [ ]:
########## Batch data fitting ##########

print('Last cell eval:', datetime.datetime.now().strftime("%H:%M on %Y-%m-%d"))

### Experimental data
fitting_df = pd.read_excel(input_file, sheet_name = 'Fitting Data')
fit_header = [i for i in fitting_df]
ndpts = len(fitting_df['Entry'])-1

### Create dataframe with conditions being tracked
conditions_df = fitting_df[[string for string in fit_header if 'Condition' in string]]
conditions = conditions_df.iloc[0].values.tolist()
conditions_df.columns = conditions
conditions_df = conditions_df.drop(index = 0)
conditions_df.index = range(len(conditions_df))

### Create dataframe feed compounds being controlled
feed_df = fitting_df[[string for string in fit_header if 'Feed' in string]]
feed_species = feed_df.iloc[0].values.tolist()
feed_df.columns = feed_species
feed_df = feed_df.drop(index = 0)
feed_df.index = range(len(feed_df))

### Create dataframe of products being measured
fitted_df = fitting_df[[string for string in fit_header if 'Final' in string]]
fitted_species = fitted_df.iloc[0].values.tolist()
fitted_df.columns = fitted_species
fitted_df = fitted_df.drop(index = 0)
fitted_df.index = range(len(fitted_df))
nspecies_modeled = len(fitted_df.columns)

### Create dataframes for initial and final concentrations
C0_df = pd.DataFrame(np.zeros([ndpts,len(species_df)]),columns = all_species)
C_expt = fitted_df.to_numpy('float')
C_expt = np.reshape(C_expt,np.prod(C_expt.shape))
log_C_expt = np.log(C_expt)

for i in feed_species:
    C0_df[i] = feed_df[i]

###### Run fitting algorithm

### Optional forcing of values; also have to set in code
k1 = 0.02087
KC1 = 0.002303
k2 = 0.00
KC2  = 12000
Kads_H2O = 0

par_names = ['k1','KC1','k2','KC2','Kads_H2O']

guess = [0.1,0.1,0.1,0.1,0.1]
npars = len(guess)

### Regression
popt,pcov = curve_fit(log_batch_predict,conditions_df,log_C_expt,guess,method='lm')

### Calculate Student's t value
alpha = 0.05 # 95% confidence interval
dof = ndpts-npars # number of degrees of freedom
tval = tdist.ppf(1.0-alpha/2.0,dof) # student t value for the dof and confidence level

### Confidence interval calculation
sigma = np.sqrt(np.diag(pcov))
ci_half = sigma*tval
ci = np.hstack((popt-ci_half,popt+ci_half))

### Create output dataframe
soln = pd.DataFrame(popt,index=par_names,columns=['Value'])
soln['95% CI Half Width'] = ci_half
soln['95% CI Half Width Rel %'] = ci_half/popt*100
#pd.options.display.float_format = "{:,.3f}".format
print('\n', soln, '\n')

### Dataframe of simulated concentrations
C_model = batch_predict(conditions_df,*popt)
C_model = np.reshape(C_model,(ndpts,nspecies_modeled))
C_model_df = pd.DataFrame(C_model,columns=fitted_species)

### Reshape experimental concentration matrix
C_expt = np.reshape(C_expt,(ndpts,nspecies_modeled))
C_expt_df = pd.DataFrame(C_expt,columns=fitted_species)

### Calculate sum of squared residuals
ssr = 0
for i in range(ndpts):    
    sqr_resid = (C_model[i]-C_expt[i])**2
    ssr = ssr + sqr_resid
print('\nSSR = ' + '{:,.3e}'.format(ssr[0]))

print('\nFitted Parameters\n\n', soln, '\n\nParity plots')

### Generate parity plots from experiment and model dataframes using defined function
fig,axs = parity_plot(C_expt_df,C_model_df)


In [ ]:
########## Integrate PBR and plot data ##########
print('Last cell eval:', datetime.datetime.now().strftime("%H:%M on %Y-%m-%d"))

### Define constants
k1  =   5E5
KC1 = 3000
k2  =   1E2
KC2 = 1000
Kads_H2O = 0

pars = [k1,KC1,k2,KC2,Kads_H2O]

### Inlet temperature
T0 = 35 + 273 # K

### Inlet gas flow rate
v0 = 50 / 1000 # sccm -> slpm (standard cubic centimers per minute -> standard liters per minute)
F0 = 1 * v0 / 0.08314 / 273.15 # moles/min with STP = 1 bar, 273.15 K

### Inlet pressure
P0 = 10      # bar

### Catalyst mass
W = 0.1     # g

### Inlet partial pressures
yj0 = np.zeros(nspecies)
yj0[IDN['acetone']]           = 0.1  # mol / mol
yj0[IDN['water']]             = 0.0  # mol / mol
yj0[IDN['inert']]             = 1-sum(yj0) # mol / mol, balance

### Inlet moles
Fj0 = yj0 * F0

### Determine integration span and spacing
tlim = (0,W)
tlist = np.linspace(tlim[0],tlim[1],num=11) # specify number of evenly-spaced points to report

### Inputs
conditions = [T0,P0,v0,Fj0]    
inputs = [conditions,pars]

### Initial conditions
U0 = [T0,1]+Fj0.tolist()

### Integrate
solution = solve_ivp(diff_PBR,tlim,U0,t_eval=tlist,args=inputs,method='Radau')
W = solution.t
U = solution.y.T
T = U[:,0]
gamma = U[:,1]
Fj = U[:,2:]
v = v0 # Assumes constant volume
P = P0 * gamma.reshape(len(gamma),1)
yj = Fj / np.sum(Fj,1).reshape(len(W),1)
Pj = yj * P

### Convert lists to numpy arrays
W = np.array(W)
Fj = np.array(Fj)

### Conversion, yield, selectivity
X = (Fj0[IDN['acetone']]*np.ones(len(Fj[:,0]))-Fj[:,IDN['acetone']])/Fj0[IDN['acetone']]
Y_DAA = 2*(Fj[:,IDN['diacetone alcohol']]-Fj0[IDN['diacetone alcohol']])/Fj0[IDN['acetone']]
Y_MO = 2*(Fj[:,IDN['mesityl oxide']]-Fj0[IDN['mesityl oxide']])/Fj0[IDN['acetone']]
S_DAA = Y_DAA / (Y_DAA + Y_MO)
S_MO  = Y_MO / (Y_DAA + Y_MO)

### Define pandas dataframes with outputs
soln_df_summary = pd.DataFrame(W,columns = ['Catalyst mass (g)'])
soln_df_summary[['X_Acetone']] = pd.DataFrame(X)
soln_df_summary[['Y_DAA']] = pd.DataFrame(Y_DAA)
soln_df_summary[['Y_MO']] = pd.DataFrame(Y_MO)
soln_df_summary[['S_DAA']] = pd.DataFrame(S_DAA)
soln_df_summary[['S_MO']] = pd.DataFrame(S_MO)

soln_df_Fj = pd.DataFrame(W,columns = ['Catalyst mass (g)'])
soln_df_Fj[['Acetone']] = pd.DataFrame(Fj[:,IDN['acetone']])
soln_df_Fj[['Diacetone alcohol']] = pd.DataFrame(Fj[:,IDN['diacetone alcohol']])
soln_df_Fj[['Mesityl oxide']] = pd.DataFrame(Fj[:,IDN['mesityl oxide']])

soln_df_Pj = pd.DataFrame(W,columns = ['Catalyst mass (g)'])
soln_df_Pj[['Acetone']] = pd.DataFrame(Pj[:,IDN['acetone']])
soln_df_Pj[['Diacetone alcohol']] = pd.DataFrame(Pj[:,IDN['diacetone alcohol']])
soln_df_Pj[['Mesityl oxide']] = pd.DataFrame(Pj[:,IDN['mesityl oxide']])

soln_df_yj = pd.DataFrame(W,columns = ['Catalyst mass (g)'])
soln_df_yj[['Acetone']] = pd.DataFrame(yj[:,IDN['acetone']])
soln_df_yj[['Diacetone alcohol']] = pd.DataFrame(yj[:,IDN['diacetone alcohol']])
soln_df_yj[['Mesityl oxide']] = pd.DataFrame(yj[:,IDN['mesityl oxide']])

### Export dataframe to Excel
with pd.ExcelWriter('PBR_model_output.xlsx') as writer:  
    soln_df_summary.to_excel(writer, sheet_name='Summary')
    soln_df_Fj.to_excel(writer, sheet_name='Fj (mol min^-1)')
    soln_df_Pj.to_excel(writer, sheet_name='Pj (bar)')
    soln_df_yj.to_excel(writer, sheet_name='yj ()')

### Report desired df
print('\n',soln_df_summary,'\n')

print('Conversion with ',max(W),'g cat is ','{0:,.3f}'.format(X[-1]*100),'%')

### Plot data
fig = plt.figure()
ax1 = plt.subplot(111)

ax1.plot(W,Fj[:,IDN['acetone']],'b')
ax1.plot(W,Fj[:,IDN['diacetone alcohol']],'-g')
ax1.plot(W,Fj[:,IDN['mesityl oxide']],'-.m')
ax1.set_xlabel('W (g)')
ax1.set_ylabel('F (mol/s)')
ax1.legend(('A','DAA','MO'))

fig.tight_layout()
plt.show()


In [ ]:
########## PBR data fitting ##########

print('Last cell eval:', datetime.datetime.now().strftime("%H:%M on %Y-%m-%d"))

### Experimental data
fitting_df = pd.read_excel(input_file, sheet_name = 'Fitting Data')
fit_header = [i for i in fitting_df]
ndpts = len(fitting_df['Entry'])-1

### Create dataframe with conditions being tracked
conditions_df = fitting_df[[string for string in fit_header if 'Condition' in string]]
conditions = conditions_df.iloc[0].values.tolist()
conditions_df.columns = conditions
conditions_df = conditions_df.drop(index = 0)
conditions_df.index = range(len(conditions_df))

### Create dataframe feed compounds being controlled
feed_df = fitting_df[[string for string in fit_header if 'Feed' in string]]
feed_species = feed_df.iloc[0].values.tolist()
feed_df.columns = feed_species
feed_df = feed_df.drop(index = 0)
feed_df.index = range(len(feed_df))

### Create dataframe of products being measured
fitted_df = fitting_df[[string for string in fit_header if 'Final' in string]]
fitted_species = fitted_df.iloc[0].values.tolist()
fitted_df.columns = fitted_species
fitted_df = fitted_df.drop(index = 0)
fitted_df.index = range(len(fitted_df))
nspecies_modeled = len(fitted_df.columns)

### Create dataframes for initial and final concentrations
yj0_df = pd.DataFrame(np.zeros([ndpts,len(species_df)]),columns = all_species)
y_expt = fitted_df.to_numpy('float')
y_expt = np.reshape(y_expt,np.prod(y_expt.shape))
log_y_expt = np.log(y_expt)

for i in feed_species:
    yj0_df[i] = feed_df[i]

yj0_df['inert'] = 1-yj0_df.sum(1)  

###### Input dataframes
input_df = pd.concat([conditions_df,yj0_df],axis=1)

###### Run fitting algorithm

### Optional forcing of values; also have to set in code
k1  =   5E5
KC1 = 3000
k2  =   1E2
KC2 = 1000
Kads_H2O = 0

par_names = ['k1','KC1','k2','KC2','Kads_H2O']

guess = [5E5,3E3,1E2,1E3,0]
npars = len(guess)

### Regression
popt,pcov = curve_fit(log_PBR_predict,input_df,log_y_expt,guess,method='lm')

### Calculate Student's t value
alpha = 0.05 # 95% confidence interval
dof = ndpts-npars # number of degrees of freedom
tval = tdist.ppf(1.0-alpha/2.0,dof) # student t value for the dof and confidence level

### Confidence interval calculation
sigma = np.sqrt(np.diag(pcov))
ci_half = sigma*tval
ci = np.hstack((popt-ci_half,popt+ci_half))

### Create output dataframe
soln = pd.DataFrame(popt,index=par_names,columns=['Value'])
soln['95% CI Half Width'] = ci_half
soln['95% CI Half Width Rel %'] = ci_half/popt*100
#pd.options.display.float_format = "{:,.3f}".format
print('\n', soln, '\n')

### Dataframe of simulated concentrations
y_model = PBR_predict(input_df,*popt)
y_model = np.reshape(y_model,(ndpts,nspecies_modeled))
y_model_df = pd.DataFrame(y_model,columns=fitted_species)

### Reshape experimental concentration matrix
y_expt = np.reshape(y_expt,(ndpts,nspecies_modeled))
y_expt_df = pd.DataFrame(y_expt,columns=fitted_species)

### Calculate sum of squared residuals
ssr = 0
for i in range(ndpts):    
    sqr_resid = (y_model[i]-y_expt[i])**2
    ssr = ssr + sqr_resid
print('\nSSR = ' + '{:,.3e}'.format(ssr[0]))

print('\nFitted Parameters\n\n', soln, '\n\nParity plots')

### Generate parity plots from experiment and model dataframes using defined function
fig,axs = parity_plot(y_expt_df,y_model_df)
